In [1]:
%load_ext autoreload

In [44]:
%autoreload 2

import numpy as np
import os
import pandas as pd
from spynal.matIO import loadmat

# Load Data

In [3]:
filename = "/scratch2/weka/millerlab/eisenaj/datasets/anesthesia/mat/propofolPuffTone/Mary-Anesthesia-20160912-02.mat"

In [7]:

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
# variables = ['electrodeInfo', 'lfpSchema', 'sessionInfo', 'trialInfo', 'unitInfo']
# session_vars, T, N, dt = load_session_data(session, all_data_dir, variables, data_class=data_class, verbose=False)
# electrode_info, lfp_schema, session_info, trial_info, unit_info = session_vars['electrodeInfo'], session_vars['lfpSchema'], session_vars['sessionInfo'], session_vars['trialInfo'], session_vars['unitInfo']

variables = ['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'trialInfo', 'unitInfo']
session_vars = {}
for arg in variables:
    session_vars[arg] = loadmat(filename, variables=[arg], verbose=True)

T = len(session_vars['lfpSchema']['index'][0])
N = len(session_vars['lfpSchema']['index'][1])
dt = session_vars['lfpSchema']['smpInterval'][0]

electrode_info, lfp, lfp_schema, session_info, spike_times, trial_info, unit_info = session_vars['electrodeInfo'], session_vars['lfp'], session_vars['lfpSchema'], session_vars['sessionInfo'], session_vars['spikeTimes'], session_vars['trialInfo'], session_vars['unitInfo']

electrodeInfo : dict with keys[NSP,Properties,area,array,channel,chnlID,electrode,file,gridLoc,hasLFP,hemisphere,numUnits,session,]

lfp : numpy.array(6504301,249,) of type float64

lfpSchema : dict with keys[dataDim,dataUnits,description,dims,history,index,indexUnits,mask,smpInterval,smpRate,timeUnits,userData,]

sessionInfo : dict with keys[NSPs,Properties,areas,arrays,drug,drugDose,drugDuration,drugEnd,drugStart,drugTask,eyesClose,eyesOpen,hasDrug,hasWakeUpTest,numCorrect,numTrials,pctBadTrials,pctCorrect,session,subject,tasks,]

spikeTimes : numpy.array(234,) of type object

trialInfo : dict with keys[Properties,badTimingTrials,badTrials,cpt_puffOff,cpt_puffOn,cpt_refTime,cpt_toneOff,cpt_toneOn,cpt_trialType,drugPhase,drugPhaseTime,refTime,session,task,trial,trialStart,]

unitInfo : dict with keys[NSP,Properties,area,array,channel,chnlID,electrode,file,gridLoc,hemisphere,numSpikes,session,unit,unitID,]



In [10]:
eyes_open = session_info['eyesOpen'][-1] if isinstance(session_info['eyesOpen'], np.ndarray) else session_info['eyesOpen']
eyes_close = session_info['eyesClose'][-1] if isinstance(session_info['eyesClose'], np.ndarray) else session_info['eyesClose']

In [11]:
section_times = dict( 
        wake=(0, session_info['drugStart'][0]),
        induction=(session_info['drugStart'][0], eyes_close),
        anesthesia=(eyes_close, session_info['drugEnd'][1]),
        recovery=(session_info['drugEnd'][1], T*dt)
)
sections = list(section_times.keys())

# Generate Trimmed Data

In [27]:
cutoff_time = 10*60 # 10 minutes by 60 seconds

In [46]:
lfp[:int(cutoff_time/dt)].nbytes/1e9

1.1952

In [47]:
lfp_trimmed = lfp[:int(cutoff_time/dt)]
spike_times_trimmed = [times[times <= cutoff_time] for times in spike_times]
unit_areas = unit_info['area']
lfp_areas = electrode_info['area']

In [48]:
save_dict = dict(
    lfp=lfp_trimmed,
    spike_times=spike_times_trimmed,
    unit_areas=unit_areas,
    lfp_areas=lfp_areas,
    dt=dt
)

In [49]:
pd.to_pickle(save_dict, os.path.join("/scratch2/weka/millerlab/eisenaj/ChaoticConsciousness/data", "data_for_mitchell.pkl"))